# MultiplEYE preprocessing

In [26]:
from pathlib import Path
import polars as pl
import pandas as pd
import re
import json
import PIL
import math
from sanity import extract_informatio_file_path
from report import load_data, check_gaze, check_metadata, preprocess, check_events, plot_gaze, report_to_file as report_meta, plot_main_sequence
from functools import partial


import pymovements as pm

In [27]:
# Create a dictionary with all the relevant information from the file path


path_asc_file = "data\\017_NL_NL_1_ET1_testrun_1733144369\\017nlnl1.asc"
local_file_part = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing"
stimulus_file_path = "C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024"
logfile_path = f"C:\\Users\saphi\PycharmProjects\multipleye-preprocessing\data\\017_NL_NL_1_ET1_testrun_1733144369\logfiles\EXPERIMENT_LOGFILE_1_017_2024-12-02_1733144369.txt"


    
vars_dict = extract_informatio_file_path(path_asc_file, local_file_part, stimulus_file_path, logfile_path)



Match was found at 5-52: 017_NL_NL_1_ET1_testrun_1733144369\017nlnl1.asc
Group 1 found at 5-8: 017
Group 2 found at 9-11: NL
Group 3 found at 12-14: NL
Group 4 found at 15-16: 1
Group 5 found at 17-20: ET1
Group 6 found at 21-39: testrun_1733144369
Group 7 found at 40-48: 017nlnl1
C:\Users\saphi\PycharmProjects\multipleye-preprocessing\quality-report\output\NL_NL_Nijmegen_1_2024
True


In [28]:
vars_dict["asc_file"]


WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/017_NL_NL_1_ET1_testrun_1733144369/017nlnl1.asc')

In [29]:
def report_to_file(message: str, report_file =vars_dict["report_file"]):
    with open(report_file, "a", encoding="utf-8") as report_file:
        report_file.write(f"{message}\n")
 

In [8]:
from stimulus import Stimulus
stimulus_dir = vars_dict["stimulus_file_path"]
lang = vars_dict["lang"].lower()
country = vars_dict["country"].lower()
labnum = vars_dict["labnum"]
stimulus_name = "PopSci_MultiplEYE"

stimulus = Stimulus.load(stimulus_dir, lang, country, labnum, stimulus_name)
for page in stimulus.pages:
    print(page.number, page.image_path)


1 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_1_nl.png
2 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_2_nl.png
3 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_3_nl.png
4 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_4_nl.png
5 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_5_nl.png
6 C:\Users\saphi\PycharmProjects\multipleye-preprocessing\data\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\stimuli_images_nl_nl_1\popsci_multipleye_id1_page_6_nl.png
7 C:\Users\saphi\Pycha

In [9]:

with open (vars_dict["json"]) as f:
    lab_config = json.load(f)
lab_config

{'Contact information': {'First_name': 'Stefan',
  'Last_name': 'Frank',
  'Affiliation': 'Radboud University',
  'Email': 'stefan.frank@ru.nl',
  'Phone': ''},
 'Data_collection_name': 'MultiplEYE_NL_NL_Nijmegen_1_2024',
 'Preregistration_core_or_additional_data_collection': 'pre-registration for core dataset',
 'Name_eye-tracker': 'EyeLink 1000 Plus',
 'Name_eye-tracker_manufacturer': 'SR Research',
 'Specification_camera_system': 'desktop mount',
 'Name_computer_operation_system': 'Windows 10',
 'Name_monitor': ['XL2430T', 'BenQ'],
 'Monitor_resolution_in_px': '(3840,2160)',
 'Screen_size_in_cm': '(59.8,33.6)',
 'Distance_in_cm': '60',
 'Script_direction': 'LTR',
 'Language_code': 'NL',
 'Country_code': 'NL',
 'Lab_number': '1',
 'City': 'Nijmegen',
 'Estimated_end_year_of_data_collection': '2024',
 'Use_of_multiple_devices': 'No',
 'Stimuli_preparation': ['Stimuli will be or has been prepared by our own lab'],
 'Psychometric_tests': {'Are_tests_conducted': 'No',
  'LWMC': '',
  'PL

## ASC to sample-level CSV

Convert the ASC files to CSV files (one for each page) where each row is a sample.

In [10]:
import importlib
stimulus_config_spec = importlib.util.spec_from_file_location(
        "stimulus_config", vars_dict["experiment_config"]
    )

stimulus_config = importlib.util.module_from_spec(stimulus_config_spec)
stimulus_config_spec.loader.exec_module(stimulus_config)

stimulus_config.IMAGE_WIDTH_PX

2375

In [11]:
from report import load_data

gaze = load_data(vars_dict['asc_file'], vars_dict["stimulus_file_path"], config=vars_dict["experiment_config"])

ModuleSpec(name='stimulus_config', loader=<_frozen_importlib_external.SourceFileLoader object at 0x0000026D03994990>, origin='C:\\Users\\saphi\\PycharmProjects\\multipleye-preprocessing\\data\\stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024\\config\\config_nl_nl_Nijmegen_1_2024.py')


In [12]:
gaze.frame

time,pupil,trial,practice,activity,stimulus,screen,pixel
i64,f64,str,bool,str,str,str,list[f64]
1335359,1178.0,"""PRACTICE_trial_1""",true,"""reading""","""Lit_NorthWind_7""","""page_1""","[1194.9, 877.0]"
1335361,1178.0,"""PRACTICE_trial_1""",true,"""reading""","""Lit_NorthWind_7""","""page_1""","[1194.3, 877.7]"
1335363,1179.0,"""PRACTICE_trial_1""",true,"""reading""","""Lit_NorthWind_7""","""page_1""","[1193.9, 877.7]"
1335365,1179.0,"""PRACTICE_trial_1""",true,"""reading""","""Lit_NorthWind_7""","""page_1""","[1194.3, 876.9]"
1335367,1180.0,"""PRACTICE_trial_1""",true,"""reading""","""Lit_NorthWind_7""","""page_1""","[1194.7, 875.7]"
…,…,…,…,…,…,…,…
6853775,1153.0,"""trial_10""",false,"""question""","""PopSci_Caveman_12""","""question_12132""","[2254.5, 1589.8]"
6853777,1153.0,"""trial_10""",false,"""question""","""PopSci_Caveman_12""","""question_12132""","[2254.5, 1588.2]"
6853779,1153.0,"""trial_10""",false,"""question""","""PopSci_Caveman_12""","""question_12132""","[2254.7, 1586.8]"


In [13]:
gaze._metadata


{'weekday': 'Mon',
 'month': 'Dec',
 'day': 2,
 'time': '00:15:26',
 'year': 2024,
 'version_1': 'EYELINK II 1',
 'version_2': 'EYELINK II CL v4.594 Jul  6 2012',
 'resolution': (2376, 1801),
 'tracking_mode': 'CR',
 'sampling_rate': 500.0,
 'file_sample_filter': '2',
 'link_sample_filter': '1',
 'tracked_eye': 'L',
 'mount_configuration': {'mount_type': 'Desktop',
  'head_stabilization': 'stabilized',
  'eyes_recorded': 'monocular',
  'short_name': 'MTABLER'},
 'pupil_data_type': 'AREA',
 'version_number': '4.594',
 'model': 'EyeLink 1000',
 'datetime': datetime.datetime(2024, 12, 2, 0, 15, 26),
 'calibrations': [{'timestamp': '1307711.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '1654534.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '2404756.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': 'LEFT'},
  {'timestamp': '2459282.0',
   'num_points': '9',
   'type': 'P-CR',
   'tracked_eye': '

In [14]:
lab_config

{'Contact information': {'First_name': 'Stefan',
  'Last_name': 'Frank',
  'Affiliation': 'Radboud University',
  'Email': 'stefan.frank@ru.nl',
  'Phone': ''},
 'Data_collection_name': 'MultiplEYE_NL_NL_Nijmegen_1_2024',
 'Preregistration_core_or_additional_data_collection': 'pre-registration for core dataset',
 'Name_eye-tracker': 'EyeLink 1000 Plus',
 'Name_eye-tracker_manufacturer': 'SR Research',
 'Specification_camera_system': 'desktop mount',
 'Name_computer_operation_system': 'Windows 10',
 'Name_monitor': ['XL2430T', 'BenQ'],
 'Monitor_resolution_in_px': '(3840,2160)',
 'Screen_size_in_cm': '(59.8,33.6)',
 'Distance_in_cm': '60',
 'Script_direction': 'LTR',
 'Language_code': 'NL',
 'Country_code': 'NL',
 'Lab_number': '1',
 'City': 'Nijmegen',
 'Estimated_end_year_of_data_collection': '2024',
 'Use_of_multiple_devices': 'No',
 'Stimuli_preparation': ['Stimuli will be or has been prepared by our own lab'],
 'Psychometric_tests': {'Are_tests_conducted': 'No',
  'LWMC': '',
  'PL

In [ ]:
vars_dict["config"]

In [15]:


report_file_2 = open(f"{vars_dict['output_dir']}/{vars_dict['participant_abbr']}_report2.txt", "w", encoding="utf-8")
report = partial(report_meta, report_file=report_file_2)
check_gaze(gaze, report)    
check_metadata(gaze._metadata, report)

In [ ]:
preprocess(gaze)


In [16]:
check_events(gaze, report)

In [17]:
plot_dir = vars_dict["output_dir"] / f"{vars_dict['participant_abbr']}_plots"
plot_dir.mkdir(exist_ok=True)

plot_main_sequence(gaze.events, plot_dir)

ValueError: There are no saccades in the event dataframe. Please make sure you ran a saccade detection algorithm. The event name should be stored in a colum called "name".

In [18]:
print(plot_dir)
print(gaze)

C:\Users\saphi\PycharmProjects\multipleye-preprocessing\quality-report\output\NL_NL_Nijmegen_1_2024\017nlnl1_plots
Experiment(sampling_rate=500.00, screen=Screen(width_px=2375, height_px=1800, width_cm=37, height_cm=28, distance_cm=60, origin=upper left), eyetracker=None)
shape: (1_896_019, 8)
┌─────────┬────────┬──────────────┬──────────┬──────────┬──────────────┬─────────────┬─────────────┐
│ time    ┆ pupil  ┆ trial        ┆ practice ┆ activity ┆ stimulus     ┆ screen      ┆ pixel       │
│ ---     ┆ ---    ┆ ---          ┆ ---      ┆ ---      ┆ ---          ┆ ---         ┆ ---         │
│ i64     ┆ f64    ┆ str          ┆ bool     ┆ str      ┆ str          ┆ str         ┆ list[f64]   │
╞═════════╪════════╪══════════════╪══════════╪══════════╪══════════════╪═════════════╪═════════════╡
│ 1335359 ┆ 1178.0 ┆ PRACTICE_tri ┆ true     ┆ reading  ┆ Lit_NorthWin ┆ page_1      ┆ [1194.9,    │
│         ┆        ┆ al_1         ┆          ┆          ┆ d_7          ┆             ┆ 877.0]      

In [19]:

logfile= pl.read_csv(logfile_path, separator="\t")

In [20]:
logfile

timestamp,trial_number,stimulus_number,page_number,screen_onset_timestamp,keypress_timestamp,key_pressed,question,answer_correct,message
f64,str,str,str,str,str,str,str,str,str
28307.5598,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** DATE:2024-12-02"""
28310.4511,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** EXP_START_TIMESTAMP:173314…"
28313.1554,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** SESSION_ID:1"""
28315.7889,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** PARTICIPANT_ID:017"""
28318.4455,"""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""<NA>""","""*** DATASET_TYPE:test_sessions"""
…,…,…,…,…,…,…,…,…,…
6.4596e6,"""10""","""12""","""12131""","""6447537.151799999""","""6459634.0001""","""left""","""True""","""False""","""FINAL ANSWER: correct answer i…"
6.4765e6,"""10""","""12""","""12132""","""6476541.324999999""","""6476537.9149""","""left""","""True""","""True""","""preliminary answer: target_key"""
6.4778e6,"""10""","""12""","""12132""","""6476541.324999999""","""6477786.0203""","""space""","""True""","""True""","""preliminary answer: final_conf…"


In [21]:
stimulus_frame = logfile.filter(
              (pl.col("stimulus_number") == f"{11}")
            )
stimulus_frame.write_csv("stimulus_frame.csv")
stimulus_frame

timestamp,trial_number,stimulus_number,page_number,screen_onset_timestamp,keypress_timestamp,key_pressed,question,answer_correct,message
f64,str,str,str,str,str,str,str,str,str
5.2262e6,"""8""","""11""","""<NA>""","""5224634.005999999""","""5226185.636599999""","""space""","""False""","""<NA>""","""optional break duration: 1551.…"
5.2933e6,"""8""","""11""","""1""","""5273527.3148""","""5293313.783299999""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
5.3138e6,"""8""","""11""","""2""","""5297724.3399""","""5313833.398999999""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
5.3237e6,"""8""","""11""","""3""","""5318289.9048999995""","""5323665.7484""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
5.3428e6,"""8""","""11""","""4""","""5327813.6411""","""5342769.763699999""","""space""","""False""","""<NA>""","""showing: Arg_PISARapaNui"""
…,…,…,…,…,…,…,…,…,…
5.6521e6,"""8""","""11""","""11131""","""5649397.033599999""","""5652097.322699999""","""space""","""True""","""False""","""preliminary answer: final_conf…"
5.6521e6,"""8""","""11""","""11131""","""5635561.3614""","""5652097.322699999""","""left""","""True""","""False""","""FINAL ANSWER: correct answer i…"
5.6653e6,"""8""","""11""","""11232""","""5665284.3851""","""5665281.4124""","""up""","""True""","""True""","""preliminary answer: target_key"""


In [22]:
from stimulus import load_stimuli
stimuli = load_stimuli(vars_dict["stimulus_file_path"], vars_dict["lang"], vars_dict["country"], vars_dict["labnum"])

def check_all_screens_logfile(logfile: pl, stimuli: Stimulus):
    for stimulus in stimuli:
        print(f"Checking {stimulus.name} in Logfile")
        stimulus_frame = logfile.filter(
              (pl.col("stimulus_number") == f"{stimulus.id}")
            )
        # check if all pages are present        
        for page in stimulus.pages:
            if f"{page.number}" not in stimulus_frame["page_number"].to_list():
                print(f"Missing page {stimulus.name} {page.number} in Logfile")  
                report_to_file(f"Missing page {stimulus.name} {page.number} in Logfile")
        # check if all questions are present              
        for question in stimulus.questions:
            if f"{question.id}" not in stimulus_frame["page_number"].to_list() and f"{question.id[1:]}" not in stimulus_frame["page_number"].to_list():
                print(f"Missing question_{question.id}")
                report_to_file(f"Missing question_{question.id} in Logfile")
                print(stimulus_frame["screen"])
      
check_all_screens_logfile(logfile, stimuli)

Checking PopSci_MultiplEYE in Logfile
Checking Ins_HumanRights in Logfile
Checking Ins_LearningMobility in Logfile
Checking Lit_Alchemist in Logfile
Checking Lit_MagicMountain in Logfile
Checking Lit_Solaris in Logfile
Checking Lit_BrokenApril in Logfile
Checking Arg_PISACowsMilk in Logfile
Checking Arg_PISARapaNui in Logfile
Checking PopSci_Caveman in Logfile
Checking Enc_WikiMoon in Logfile
Checking Lit_NorthWind in Logfile


In [23]:
## check if all pages screens ect are present

from stimulus import load_stimuli
stimuli = load_stimuli(vars_dict["stimulus_file_path"], vars_dict["lang"], vars_dict["country"], vars_dict["labnum"])

def check_all_screens(gaze, stimuli):
    for stimulus in stimuli:
        print(f"Checking {stimulus.name}")
        stimulus_frame = gaze.frame.filter(
              (pl.col("stimulus") == f"{stimulus.name}_{stimulus.id}")
            ).unique("screen")
        # check if all pages are present        
        for page in stimulus.pages:
            if f"page_{page.number}" not in stimulus_frame["screen"].to_list():
                print(f"Missing page {page.number}")  
                report_to_file(f"Missing page {page.number} in asc file")
        # check if all questions are present              
        for question in stimulus.questions:
            if f"question_{question.id}" not in stimulus_frame["screen"].to_list() and f"question_{question.id[1:]}" not in stimulus_frame["screen"].to_list():
                print(f"Missing question_{question.id}")
                report_to_file(f"Missing question_{question.id} in asc file")
                print(stimulus_frame["screen"])
      
check_all_screens(gaze, stimuli)

Checking PopSci_MultiplEYE
Checking Ins_HumanRights
Checking Ins_LearningMobility
Checking Lit_Alchemist
Checking Lit_MagicMountain
Checking Lit_Solaris
Checking Lit_BrokenApril
Checking Arg_PISACowsMilk
Checking Arg_PISARapaNui
Checking PopSci_Caveman
Checking Enc_WikiMoon
Checking Lit_NorthWind


In [24]:
def load_messages(asc_file: Path, REGEX):
    with open(asc_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
    messages = []
    for line in lines:
        match = re.match(REGEX, line)
        if match:
            messages.append(match.groupdict())
    return messages

REGEX = r'MSG\s+(?P<timestamp>\d+[.]?\d*)\s+(?P<message>\w+)'


messages = load_messages(vars_dict["asc_file"], REGEX)

In [25]:
print(messages)

[{'timestamp': '404044.0', 'message': 'var'}, {'timestamp': '404096.0', 'message': 'welcome_screen'}, {'timestamp': '1124828.0', 'message': 'informed_consent_screen'}, {'timestamp': '1129138.0', 'message': 'start_experiment'}, {'timestamp': '1129138.0', 'message': 'stimulus_order_version'}, {'timestamp': '1129140.0', 'message': 'showing_instruction_screen_1'}, {'timestamp': '1131992.0', 'message': 'showing_instruction_screen_2'}, {'timestamp': '1151361.0', 'message': 'showing_instruction_screen_3'}, {'timestamp': '1160379.0', 'message': 'camera_setup_screen'}, {'timestamp': '1172095.0', 'message': 'DISPLAY_COORDS'}, {'timestamp': '1270732.0', 'message': 'practice_text_starting_screen'}, {'timestamp': '1284445.0', 'message': 'TRIAL_VAR_LABELS'}, {'timestamp': '1284445.0', 'message': 'V_TRIAL_GROUPING'}, {'timestamp': '1284446.0', 'message': 'TRIALID'}, {'timestamp': '1285412.0', 'message': 'RECCFG'}, {'timestamp': '1285412.0', 'message': 'ELCLCFG'}, {'timestamp': '1285412.0', 'message':

In [26]:
stimulus = stimuli[2]
stimulus.name, stimulus.pages[0]

('Ins_LearningMobility',
 StimulusPage(number=1, text='VERSLAG VAN DE COMMISSIE AAN DE RAAD: Voortgangsverslag over een benchmark betreffende leermobiliteit\r\n\r\n1. INLEIDING\r\nIn de recente mededeling van de Commissie "Investeren in de jongeren van Europa" wordt het positieve effect van leermobiliteit op inzetbaarheid en actief burgerschap benadrukt. Leren, studeren en een opleiding volgen in een ander land is een unieke ervaring en kan nieuwe horizonten openen. ', image_path=WindowsPath('C:/Users/saphi/PycharmProjects/multipleye-preprocessing/data/stimuli_MultiplEYE_NL_NL_Nijmegen_1_2024/stimuli_images_NL_NL_1/ins_learningmobility_id3_page_1_NL.png')))

In [27]:
# check practice trials
def check_instructions(messages, stimuli):
    messages_only = [d.get('message') for d in messages]
    one_time_screens = ['welcome_screen', 'informed_consent_screen', 'start_experiment', 'stimulus_order_version', 'showing_instruction_screen_1', 'showing_instruction_screen_2', 'showing_instruction_screen_3', 'camera_setup_screen', 'practice_text_starting_screen', 'final_validation', 'show_final_screen', 'obligatory_break', 'obligatory_break_end']
    optional_screens = ['empty_screen', 'optional_break_screen']
    
    for stimulus in stimuli:
       # print(stimulus.instructions)
        print(f"Checking {stimulus.name}")
        for instruction in ["familiarity_rating_screen_1", "familiarity_rating_screen_2"]:
            if f"showing_{instruction}" not in [d.get('message') for d in messages]:
                print(f"Missing instruction {instruction}")
                report_to_file(f"Missing instruction {instruction} in asc file")
            else:
                print(f"Instruction {instruction} found")
        for instruction in ["practice_trial_screen_1", "practice_trial_screen_2"]:
            if f"showing_{instruction}" not in messages_only:
                print(f"Missing instruction {instruction}")
                report_to_file(f"Missing instruction {instruction} in asc file")
            else:
                print(f"Instruction {instruction} found")
        for instruction in ["start_recording_trial_stimulus", "end_recording_trial_stimulus"]:
            if f"{instruction}" not in messages_only:
                print(f"Missing screen {instruction}")
            else:
                print(f"{instruction} found")
                
    for one_time_screen in one_time_screens:
        if f"{one_time_screen}" not in messages_only:
            print(f"Missing screen {one_time_screen}")
            report_to_file(f"Missing screen {one_time_screen} in asc file")
        else:
            print(f"{one_time_screen} found")
    for optional_screen in optional_screens:
        if f"{optional_screen}" in messages_only:
            report_to_file(f"Optional screen {optional_screen} found in asc file")
            print(f"{optional_screen} found")

check_instructions(messages, stimuli)

Checking PopSci_MultiplEYE
Instruction familiarity_rating_screen_1 found
Instruction familiarity_rating_screen_2 found
Missing instruction practice_trial_screen_1
Missing instruction practice_trial_screen_2
Missing screen start_recording_trial_stimulus
Missing screen end_recording_trial_stimulus
Checking Ins_HumanRights
Instruction familiarity_rating_screen_1 found
Instruction familiarity_rating_screen_2 found
Missing instruction practice_trial_screen_1
Missing instruction practice_trial_screen_2
Missing screen start_recording_trial_stimulus
Missing screen end_recording_trial_stimulus
Checking Ins_LearningMobility
Instruction familiarity_rating_screen_1 found
Instruction familiarity_rating_screen_2 found
Missing instruction practice_trial_screen_1
Missing instruction practice_trial_screen_2
Missing screen start_recording_trial_stimulus
Missing screen end_recording_trial_stimulus
Checking Lit_Alchemist
Instruction familiarity_rating_screen_1 found
Instruction familiarity_rating_screen_2

In [28]:
messages_only = [d.get('message') for d in messages]
print(messages_only)

['var', 'welcome_screen', 'informed_consent_screen', 'start_experiment', 'stimulus_order_version', 'showing_instruction_screen_1', 'showing_instruction_screen_2', 'showing_instruction_screen_3', 'camera_setup_screen', 'DISPLAY_COORDS', 'practice_text_starting_screen', 'TRIAL_VAR_LABELS', 'V_TRIAL_GROUPING', 'TRIALID', 'RECCFG', 'ELCLCFG', 'GAZE_COORDS', 'THRESHOLDS', 'ELCL_PROC', 'ELCL_PCR_PARAM', 'fixation_trigger', 'fixation_trigger', 'DISPLAY_COORDS', 'VALIDATE', 'VALIDATE', 'VALIDATE', 'VALIDATE', 'VALIDATE', 'VALIDATE', 'VALIDATE', 'VALIDATE', 'VALIDATE', 'start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1', 'RECCFG', 'ELCLCFG', 'GAZE_COORDS', 'THRESHOLDS', 'ELCL_PROC', 'ELCL_PCR_PARAM', 'page_screen_image_onset', 'page_screen_image_offset', 'stop_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_page_1', 'RECCFG', 'ELCLCFG', 'GAZE_COORDS', 'THRESHOLDS', 'ELCL_PROC', 'ELCL_PCR_PARAM', 'fixation_trigger', 'start_recording_PRACTICE_trial_1_stimulus_Lit_NorthWind_7_pa

In [29]:
for instruction in stimuli[0].instructions:
    print(instruction.name)

welcome_screen
empty_screen
fixation_screen
obligatory_break_screen
optional_break_screen
final_screen
practice_screen
transition_screen
informed_consent_screen
instruction_screen_2
instruction_screen_3
instruction_screen_1
subject_difficulty_screen
familiarity_rating_screen_1
familiarity_rating_screen_2
camera_setup_screen
validation_screen
recalibration_screen
